# Lending Club Dataset Analysis & Preprocessing

This notebook analyzes the accepted and rejected loan datasets and performs preprocessing steps to produce cleaned outputs for downstream modeling.

## 1. Load Dataset A and Dataset B
Load both datasets from files into DataFrames and display a preview.

In [1]:
import pandas as pd
import numpy as np

accepted_path = "accepted_2007_to_2018Q4.csv"
rejected_path = "rejected_2007_to_2018Q4.csv"

accepted_df = pd.read_csv(accepted_path)
rejected_df = pd.read_csv(rejected_path)

print("Accepted shape:", accepted_df.shape)
print("Rejected shape:", rejected_df.shape)

accepted_df.head()

C:\Users\MAYANI\AppData\Local\Temp\ipykernel_21064\366661902.py:7: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted_df = pd.read_csv(accepted_path)


Accepted shape: (2260701, 151)
Rejected shape: (27648741, 9)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
rejected_df.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


## 2. Inspect Schema, Shapes, and Types
Check row/column counts, data types, and unique identifiers for each dataset.

In [3]:
def schema_overview(df: pd.DataFrame, name: str) -> None:
    print(f"\n{name} overview")
    print("-" * 40)
    print("Shape:", df.shape)
    print("Columns:", df.columns.tolist()[:20], "..." if len(df.columns) > 20 else "")
    print("Dtypes:\n", df.dtypes.value_counts())
    
    # Common identifiers if present
    id_candidates = [c for c in df.columns if c.lower() in {"id", "member_id", "loan_id", "application_id"}]
    if id_candidates:
        for col in id_candidates:
            print(f"Unique values in {col}:", df[col].nunique())

schema_overview(accepted_df, "Accepted")
schema_overview(rejected_df, "Rejected")


Accepted overview
----------------------------------------
Shape: (2260701, 151)
Columns: ['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc'] ...
Dtypes:
 float64    113
object      38
Name: count, dtype: int64
Unique values in id: 2260701
Unique values in member_id: 0

Rejected overview
----------------------------------------
Shape: (27648741, 9)
Columns: ['Amount Requested', 'Application Date', 'Loan Title', 'Risk_Score', 'Debt-To-Income Ratio', 'Zip Code', 'State', 'Employment Length', 'Policy Code'] 
Dtypes:
 object     6
float64    3
Name: count, dtype: int64


## 3. Profile Missing Values and Basic Stats
Compute missing value counts and basic descriptive statistics for both datasets.

In [4]:
def missing_profile(df: pd.DataFrame, name: str, top_n: int = 20) -> None:
    missing = df.isnull().sum().sort_values(ascending=False)
    missing_pct = (missing / len(df)) * 100
    profile = pd.DataFrame({"missing": missing, "missing_pct": missing_pct})
    profile = profile[profile["missing"] > 0]
    print(f"\n{name} missing values (top {top_n})")
    print("-" * 40)
    print(profile.head(top_n))

missing_profile(accepted_df, "Accepted")
missing_profile(rejected_df, "Rejected")

print("\nAccepted numeric stats")
print(accepted_df.describe(include=[np.number]).T.head(10))

print("\nRejected numeric stats")
print(rejected_df.describe(include=[np.number]).T.head(10))


Accepted missing values (top 20)
----------------------------------------
                                            missing  missing_pct
member_id                                   2260701   100.000000
orig_projected_additional_accrued_interest  2252050    99.617331
hardship_reason                             2249784    99.517097
hardship_payoff_balance_amount              2249784    99.517097
hardship_last_payment_amount                2249784    99.517097
payment_plan_start_date                     2249784    99.517097
hardship_type                               2249784    99.517097
hardship_status                             2249784    99.517097
hardship_start_date                         2249784    99.517097
deferral_term                               2249784    99.517097
hardship_amount                             2249784    99.517097
hardship_dpd                                2249784    99.517097
hardship_loan_status                        2249784    99.517097
hardship_length

## 4. Clean and Standardize Columns
Standardize column names, trim whitespace, and fix inconsistent formats in both datasets.

In [5]:
import re


def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(r"[^0-9a-zA-Z]+", "_", regex=True)
        .str.strip("_")
    )
    return df


def clean_common_fields(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # zip_code: keep first 3 digits if present
    if "zip_code" in df.columns:
        df["zip_code"] = (
            df["zip_code"].astype(str)
            .str.extract(r"(\d{3})", expand=False)
        )

    # emp_length: normalize text
    if "emp_length" in df.columns:
        df["emp_length"] = (
            df["emp_length"].astype(str)
            .str.strip()
            .replace({"n/a": np.nan, "None": np.nan, "nan": np.nan})
        )

    # dti: remove % if present
    if "dti" in df.columns:
        df["dti"] = (
            df["dti"].astype(str)
            .str.replace("%", "", regex=False)
        )
        df["dti"] = pd.to_numeric(df["dti"], errors="coerce")

    # annual_inc: numeric
    if "annual_inc" in df.columns:
        df["annual_inc"] = pd.to_numeric(df["annual_inc"], errors="coerce")

    # risk score variants (keep as numeric if present)
    for col in ["risk_score", "fico_range_low", "fico_range_high"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    return df


accepted_std = clean_common_fields(standardize_columns(accepted_df))
rejected_std = clean_common_fields(standardize_columns(rejected_df))

print("Accepted standardized columns:", accepted_std.columns.tolist()[:20], "...")
print("Rejected standardized columns:", rejected_std.columns.tolist()[:20], "...")

Accepted standardized columns: ['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc'] ...
Rejected standardized columns: ['amount_requested', 'application_date', 'loan_title', 'risk_score', 'debt_to_income_ratio', 'zip_code', 'state', 'employment_length', 'policy_code'] ...


## 5. Handle Missing Data
Apply imputation or row/column filtering strategies consistently across both datasets.

In [6]:
def impute_missing(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    num_cols = df.select_dtypes(include=[np.number]).columns
    cat_cols = df.select_dtypes(exclude=[np.number]).columns

    for col in num_cols:
        df[col] = df[col].fillna(df[col].median())

    for col in cat_cols:
        mode_val = df[col].mode(dropna=True)
        df[col] = df[col].fillna(mode_val.iloc[0] if not mode_val.empty else "UNKNOWN")

    return df

accepted_imputed = impute_missing(accepted_std)
rejected_imputed = impute_missing(rejected_std)

print("Missing after imputation (Accepted):", accepted_imputed.isnull().sum().sum())
print("Missing after imputation (Rejected):", rejected_imputed.isnull().sum().sum())

C:\Users\MAYANI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Missing after imputation (Accepted): 2260701
Missing after imputation (Rejected): 0


## 6. Encode Categorical Features
Encode categorical columns using one-hot encoding.

In [10]:
import re

def select_categorical_columns(accepted: pd.DataFrame, rejected: pd.DataFrame, max_unique: int = 50):
    accepted_cat = accepted.select_dtypes(exclude=[np.number]).columns
    rejected_cat = rejected.select_dtypes(exclude=[np.number]).columns
    all_cat = sorted(set(accepted_cat) | set(rejected_cat))

    keep = []
    dropped = []

    for col in all_cat:
        a_nunique = accepted[col].nunique(dropna=True) if col in accepted.columns else 0
        r_nunique = rejected[col].nunique(dropna=True) if col in rejected.columns else 0
        max_nunique = max(a_nunique, r_nunique)

        # Drop high-cardinality or identifier-like columns
        if max_nunique > max_unique or re.search(r"(^id$|_id$|member|loan|application|url|title|desc|emp_title|zip_code)", col):
            dropped.append((col, max_nunique))
        else:
            keep.append(col)

    return keep, dropped


def encode_with_limits(df: pd.DataFrame, cat_cols):
    num_df = df.select_dtypes(include=[np.number])
    existing_cat_cols = [c for c in cat_cols if c in df.columns]
    cat_df = df[existing_cat_cols]
    encoded_cat = pd.get_dummies(cat_df, drop_first=False, sparse=True, dtype=np.int8)
    return pd.concat([num_df, encoded_cat], axis=1)


cat_cols, dropped_cols = select_categorical_columns(accepted_imputed, rejected_imputed, max_unique=50)
print("Categorical columns kept:", len(cat_cols))
print("Dropped columns (name, max_unique) top 10:", dropped_cols[:10])

accepted_encoded = encode_with_limits(accepted_imputed, cat_cols)
rejected_encoded = encode_with_limits(rejected_imputed, cat_cols)

# Keep only common columns to avoid huge memory usage
common_cols = sorted(set(accepted_encoded.columns) & set(rejected_encoded.columns))
accepted_encoded = accepted_encoded[common_cols]
rejected_encoded = rejected_encoded[common_cols]

print("Encoded Accepted shape:", accepted_encoded.shape)
print("Encoded Rejected shape:", rejected_encoded.shape)

Categorical columns kept: 21
Dropped columns (name, max_unique) top 10: [('addr_state', 51), ('application_date', 4238), ('application_type', 2), ('debt_settlement_flag_date', 83), ('debt_to_income_ratio', 126145), ('desc', 124500), ('earliest_cr_line', 754), ('emp_title', 512694), ('hardship_loan_status', 5), ('id', 2260701)]
Encoded Accepted shape: (2260701, 1)
Encoded Rejected shape: (27648741, 1)


## 7. Scale/Normalize Numerical Features
Scale numerical features to prepare for downstream modeling.

In [12]:
from sklearn.preprocessing import StandardScaler

# Identify numeric columns and keep only common numeric columns
accepted_num_cols = set(accepted_imputed.select_dtypes(include=[np.number]).columns.tolist())
rejected_num_cols = set(rejected_imputed.select_dtypes(include=[np.number]).columns.tolist())
common_numeric_cols = sorted(accepted_num_cols & rejected_num_cols)

scaler = StandardScaler()
combined_numeric = pd.concat([
    accepted_imputed[common_numeric_cols],
    rejected_imputed[common_numeric_cols]
], axis=0)

scaler.fit(combined_numeric)

accepted_scaled = accepted_encoded.copy()
rejected_scaled = rejected_encoded.copy()

accepted_scaled[common_numeric_cols] = scaler.transform(accepted_imputed[common_numeric_cols])
rejected_scaled[common_numeric_cols] = scaler.transform(rejected_imputed[common_numeric_cols])

print("Scaled Accepted shape:", accepted_scaled.shape)
print("Scaled Rejected shape:", rejected_scaled.shape)

Scaled Accepted shape: (2260701, 1)
Scaled Rejected shape: (27648741, 1)


## 8. Save Preprocessed Outputs
Export the cleaned and preprocessed datasets to new files for later use.

In [13]:
accepted_scaled.to_csv("cleaned_accepted.csv", index=False)
rejected_scaled.to_csv("cleaned_rejected.csv", index=False)

print("Saved cleaned_accepted.csv and cleaned_rejected.csv")

Saved cleaned_accepted.csv and cleaned_rejected.csv
